In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

BASE_URL = "http://darksouls.wikidot.com"
WEAPONS_URL = f"{BASE_URL}/weapons"

browser = "chrome"

USER_AGENTS = {
    "chrome": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "safari": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15"
}

headers = {
    "User-Agent": USER_AGENTS[browser]
}


# Find the table block that holds both columns


In [4]:
# 🧠 Helper: Extract in-game description and availability
def scrape_weapon_detail(weapon_url):
    try:
        res = requests.get(weapon_url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.content, 'html.parser')
        content = soup.find('div', {'id': 'page-content'})

        # -------- In-Game Description --------
        in_game_description = ""
        found_description = False
        for tag in content.find_all():
            if tag.name in ['h2', 'h3'] and 'in-game description' in tag.text.lower():
                found_description = True
                continue
            if found_description:
                if tag.name in ['h2', 'h3']:
                    break
                if tag.name == 'p' and tag.find('em'):
                    in_game_description += tag.get_text(strip=True) + " "
        in_game_description = in_game_description.strip()

        # -------- Availability --------
        availability = []
        for header in content.find_all(['h2', 'h3']):
            if 'availability' in header.text.lower():
                next_ul = header.find_next('ul')
                next_p = header.find_next('p')
                if next_ul:
                    availability = [li.get_text(strip=True) for li in next_ul.find_all('li')]
                elif next_p:
                    availability = [next_p.get_text(strip=True)]
                break

        # -------- Stats Table (up to Stability) --------
        stats_table = {}
        full_table = content.find('table', class_='wiki-content-table')
        if full_table:
            header_row = full_table.find_all('tr')[0].find_all('th')
            value_row = full_table.find_all('tr')[1].find_all('td')

            for th, td in zip(header_row, value_row):
                label = th.get_text(strip=True)
                value = td.get_text(strip=True)

                if label.lower().startswith("aux") or label.lower().startswith("frampt"):
                    break  # Stop before "Aux Effects" or any extra columns
                stats_table[label] = value

        return {
            "in_game_description": in_game_description,
            "availability": availability,
            "stats_table": stats_table
        }

    except Exception as e:
        print(f"⚠️ Error scraping {weapon_url}: {e}")
        return {
            "in_game_description": "",
            "availability": [],
            "stats_table": {}
        }
# 🚀 Main scraper
def scrape_weapons_by_category():
    response = requests.get(WEAPONS_URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find('div', {'id': 'page-content'})
    table_cells = content.find_all('td')

    td_weapons = table_cells[1]
    all_tags = td_weapons.find_all(['h3', 'p'])

    weapon_data = []
    current_category = None
    current_weapons = []

    for tag in all_tags:
        if tag.name == 'h3':
            if current_category and current_weapons:
                weapon_data.append({
                    'category': current_category,
                    'weapons': current_weapons
                })
            current_category = tag.text.strip()
            current_weapons = []

        elif tag.name == 'p' and current_category:
            for a in tag.find_all('a'):
                weapon_name = a.text.strip()
                weapon_url = BASE_URL + a['href']
                print(f"🔍 Scraping {weapon_name}...")

                weapon_details = scrape_weapon_detail(weapon_url)
                time.sleep(1)  # Respectful delay

                current_weapons.append({
                    "name": weapon_name,
                    "url": weapon_url,
                    "in_game_description": weapon_details['in_game_description'],
                    "availability": weapon_details['availability'],
                    "stats_table": weapon_details['stats_table']
                })

    if current_category and current_weapons:
        weapon_data.append({
            'category': current_category,
            'weapons': current_weapons
        })

    return weapon_data

# 🔄 Run everything
weapons_full_data = scrape_weapons_by_category()


🔍 Scraping Caestus...
🔍 Scraping Claw...
🔍 Scraping Dark Hand...
🔍 Scraping Dragon Bone Fist...
🔍 Scraping Blacksmith Giant Hammer...
🔍 Scraping Blacksmith Hammer...
🔍 Scraping Hammer of Vamos...
🔍 Scraping Club...
🔍 Scraping Mace...
🔍 Scraping Morning Star...
🔍 Scraping Pickaxe...
🔍 Scraping Reinforced Club...
🔍 Scraping Warpick...
🔍 Scraping Demon's Great Hammer...
🔍 Scraping Dragon Tooth...
🔍 Scraping Grant...
🔍 Scraping Great Club...
🔍 Scraping Large Club...
🔍 Scraping Smough's Hammer...
🔍 Scraping Battle Axe...
🔍 Scraping Butcher Knife...
🔍 Scraping Crescent Axe...
🔍 Scraping Gargoyle Tail Axe...
🔍 Scraping Golem Axe...
🔍 Scraping Hand Axe...
🔍 Scraping Black Knight Greataxe...
🔍 Scraping Demon's Greataxe...
🔍 Scraping Dragon King Greataxe...
🔍 Scraping Greataxe...
🔍 Scraping Stone Greataxe...
🔍 Scraping Bandit's Knife...
🔍 Scraping Dagger...
🔍 Scraping Dark Silver Tracer...
🔍 Scraping Ghost Blade...
🔍 Scraping Parrying Dagger...
🔍 Scraping Priscilla's Dagger...
🔍 Scraping Estoc..

In [6]:
with open("dark_souls_weapons_full.json", "w") as f:
    json.dump(weapons_full_data, f, indent=2)